Connect to MongoDB in order to build a database of bills to access and work with later

In [ ]:
from pymongo import MongoClient
config = {
    'host': 'xx.xxx.xxx.xx:xxxx/x',
    'username': 'xxxxxxxx',
    'password': 'xxxxxxxx',
    'authSource': 'cool'
}

client = MongoClient(**config)
db = client.cool

In [7]:
from bs4 import BeautifulSoup
import requests

In [8]:
import re
def get_goods(soup, i, congress):
    '''
    This function takes in html soup for a given bill from Congress.gov 
    and scrapes all the helpful information from that page.
    The _id will simply be the bill number rather than a Mongo generated id.
    '''
    _id = i
    text = soup.find(id='billTextContainer').text
    date = soup.find(text=re.compile('\d+/\d+/\d+'))
    date = re.sub('[^0-9/]', '', date)
    track = soup.find(text=re.compile('This bill has the status'))[25:]
    sponsor = soup.find(text=re.compile('Rep\.\s'))
    name = re.search('[A-Z][a-z]+,\s[A-Za-z. ]+', sponsor).group()
    party = re.search('(?<=\[).', sponsor).group()
    state = re.search('(?<=-)..', sponsor).group()
    bill_dict = {'_id': i, 'congress': congress, 'date' : date, 'track': track, 
                 'name': name, 'party': party, 'state': state, 'bill': text}
    return bill_dict

This loop calls every bill number from the 115th Congress, forms the url for that bill, and puts the html into BeautifulSoup. The get_goods function collects all the information that's important and inserts a dictionary into the Mongo Database.

For the 116th Congress, I simply change 115th to 116th in both the url and the get_goods input and add 7401 to i, again for the function input. In total, it took about 5 hours to collect 15,100 bills all written in the past four years.

In [ ]:
for i in range(1, 7402):
    url = f"https://www.congress.gov/bill/115th-congress/house-bill/{i}/text?format=txt&r={i}&s=7"
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'lxml')
    try:
        db.bills.insert_one(get_goods(soup, i, '115th'))
    except:
        print(i)
    if i%500==0:
        print(i)

In [42]:
db.bills.count_documents({})

15105